In [1]:

import os
import torch
import pickle
import random
import logging
import heartpy as hp
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from os import path
import pandas as pd
from tqdm import tqdm
from glob import glob
from pyedflib import highlevel as reader
from matplotlib import pyplot as plt
from src.datasets import SessionMeta,RPPG

/home/od/anaconda3/envs/dfd-clip/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# #read an edf file
# signals, signal_headers, header = reader.read_edf('/storage/HCI/Sessions/59/Part_1_Trial18_taggingImages1.bdf')
# signals.shape

In [3]:
# signal_labels = [x["label"] for x in signal_headers]
# print(signal_labels,f"len:{len(signal_labels)}")
# print("=============================")
# print(signal_labels.index("EXG1"))
# print(signal_labels.index("EXG2"))
# print(signal_labels.index("EXG3"))

In [4]:
# signal_headers

In [5]:
# hr_freq = signal_headers[32]["sample_frequency"]
# print(hr_freq)
# hr_skip = int(hr_freq * 5)
# hr_dur = int(hr_freq *10)
# hr_data = signals[33][hr_skip:hr_skip + hr_dur]
# plt.plot([i/hr_freq for i in range(len(hr_data))],hr_data)

In [6]:
# event_freq = signal_headers[46]["sample_frequency"]
# event_skip = int(event_freq * 29)
# event_data = signals[46][event_skip:-event_skip]
# plt.plot([i/event_freq for i in range(len(event_data))],event_data)

In [7]:
c = RPPG.get_default_config()
c.train_ratio = 1.0
c.compressions = ["c23"]
# c.meta_folder = None

# logging.basicConfig(level="INFO")
logging.basicConfig(level="DEBUG")
x = RPPG(c,50,10)

INFO:root:Meta folder specified, loading meta infos....
3810it [00:00, 62735.00it/s]
INFO:root:Session meta created.
DEBUG:root:Number of session metas before checking: 3810
INFO:root:Session meta filtered.
DEBUG:root:Current number of sessions: 3136
INFO:root:Session measures loaded.
DEBUG:root:Current number of sessions: 3037
DEBUG:root:Number of session metas ready for training: 3037


In [8]:
print(len(x))
print(len(x.session_metas))

5519
3037


In [ ]:
# for i in range(10):
#     idx = random.randrange(0,len(x))
#     r = x.get_dict(idx)
#     frames,label, masks = r["frames"],r["label"],r["mask"]
#     print(f"{idx} new",np.argmax(label)+41)
#     r = x.get_dict(idx,True)
#     frames,label, masks = r["frames"],r["label"],r["mask"]
#     print(f"{idx} old",np.argmax(label)+41)

In [ ]:
# plt.figure(figsize=(100,150))
# plt.imshow(np.stack(frames.numpy().transpose((0,2,3,1)),axis=1).reshape((150,-1,3)))

In [ ]:
# plt.plot(hr_data)

In [9]:
r = x.get_dict(118,runtime=False)
measure = r["measure"]
# hr_data = r["hr_data"]
print(np.argmax(r["label"]))
# plt.plot(list(range(len(hr_data))),hr_data)


DEBUG:root:session idx:66
ERROR:root:Error occur: 
ERROR:root:Traceback (most recent call last):
  File "/home/od/Desktop/repos/dfd-clip/src/datasets.py", line 493, in get_dict
    measure_idx =  next(i for i,x in enumerate(session_measure["idx"]) if  hr_sample_end <= x)
StopIteration

DEBUG:root:session idx:967
DEBUG:root:Loading video: ./datasets/hci/cropped_faces/c23/3646/P29-Rec1-2009.09.03.11.51.42_C1 trigger _C_Section_6.avi, sample offset:2440


12803 12800
20480 26112
tensor(27)


In [ ]:

for i in tqdm(range(len(x))):
    try:
        x[i]
    except Exception as e:
        print(i,e)

In [ ]:
# # dataset consistency
# rng = random.Random()
# rng.seed(777)
# session_dirs = sorted(glob(path.join('./datasets/hci/',"Sessions","*")))
# rng.shuffle(session_dirs)
# session_metas = [None for _ in range(len(session_dirs))]
# for i,session_dir in tqdm(enumerate(session_dirs)):
#     try:
#         with open(path.join(session_dir.replace("Sessions","Metas"),"meta.pickle"),"rb") as file:
#             session_metas[i] = pickle.load(file)
#     except:
#         print(f"Error Loading: {session_dir}.")
# session_metas = [
#     meta for meta in session_metas 
#     if meta and meta.require_check(
#         video=True,
#         bdf=True,
#         time=True,
#         video_folders=[]
#     )
# ]
# len(session_metas)